In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt

%matplotlib inline

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8771873623121003610
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 5902019775087329143
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
num_classes = 120
batch_size = 10

train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [4]:
input_shape = (224, 224, 3)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [6]:
from keras.applications import VGG16
# Load the VGG model
# choosing VG166 architecture initially as my hand-built models with 3x3 filters perform the best and generally is more popular

vgg16_base = Sequential()
vgg16_base.add(VGG16(weights='imagenet',
                  include_top=False,
                pooling='avg'))
vgg16_base.add(Dense(num_classes, activation='softmax'))
vgg16_base.layers[0].trainable = False

# compile
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
vgg16_base.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(vgg16_base.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 14,776,248
Trainable params: 61,560
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [10]:
vgg16_base.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    validation_steps=200,
                    callbacks=[early_stopping, tensorboard])

Epoch 1/10
800/800 [==============================] - 382s 478ms/step - loss: 5.0060 - acc: 0.0078 - val_loss: 4.9803 - val_acc: 0.0092
Epoch 2/10
800/800 [==============================] - 380s 475ms/step - loss: 4.9684 - acc: 0.0075 - val_loss: 4.9534 - val_acc: 0.0092
Epoch 3/10
800/800 [==============================] - 382s 477ms/step - loss: 4.9437 - acc: 0.0086 - val_loss: 4.9312 - val_acc: 0.0104
Epoch 4/10
800/800 [==============================] - 379s 474ms/step - loss: 4.9236 - acc: 0.0078 - val_loss: 4.9120 - val_acc: 0.0108
Epoch 5/10
800/800 [==============================] - 385s 481ms/step - loss: 4.9071 - acc: 0.0082 - val_loss: 4.8955 - val_acc: 0.0117
Epoch 6/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8909 - acc: 0.0089 - val_loss: 4.8810 - val_acc: 0.0121
Epoch 7/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8763 - acc: 0.0081 - val_loss: 4.8680 - val_acc: 0.0113
Epoch 8/10
800/800 [============================

In [11]:
vgg16_base.save('vgg16_base.h5')

In [16]:
base_scores = vgg16_base.evaluate_generator(test_generator, steps=25, max_queue_size=10)
print("Accuracy: %.2f%%" % (base_scores[1]*100))

Accuracy: 0.00%


In [7]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization

In [8]:
base_model = VGG16(include_top=False, input_shape=input_shape)
from keras.layers import GlobalAveragePooling2D

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)

# let's add a fully-connected layer

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)


predictions = Dense(num_classes, activation='softmax')(x)


model = Model(input=base_model.input, output=predictions)

# train only the top layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [11]:
model.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 337s 421ms/step - loss: 5.1689 - acc: 0.0075 - val_loss: 4.8430 - val_acc: 0.0108
Epoch 2/10
800/800 [==============================] - 335s 419ms/step - loss: 5.0327 - acc: 0.0075 - val_loss: 4.8252 - val_acc: 0.0104
Epoch 3/10
800/800 [==============================] - 339s 424ms/step - loss: 4.9732 - acc: 0.0079 - val_loss: 4.8169 - val_acc: 0.0092
Epoch 4/10
800/800 [==============================] - 336s 420ms/step - loss: 4.9408 - acc: 0.0095 - val_loss: 4.8082 - val_acc: 0.0117
Epoch 5/10
800/800 [==============================] - 336s 420ms/step - loss: 4.9290 - acc: 0.0074 - val_loss: 4.8038 - val_acc: 0.0088
Epoch 6/10
800/800 [==============================] - 338s 422ms/step - loss: 4.9110 - acc: 0.0096 - val_loss: 4.7982 - val_acc: 0.0138
Epoch 7/10
800/800 [==============================] - 336s 421ms/step - loss: 4.8971 - acc: 0.0080 - val_loss: 4.7976 - val_acc: 0.0108
Epoch 8/10
800/800 [============================

In [13]:
# continue training vgg16_base

from keras.models import load_model
vgg16_base = load_model('vgg16_base.h5')

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [15]:
vgg16_base.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    callbacks=[early_stopping])

Epoch 1/10
  2/800 [..............................] - ETA: 19:08 - loss: 5.0445 - acc: 0.0000e+00

ResourceExhaustedError: OOM when allocating tensor with shape[10,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/Adam/gradients/vgg16_2/block1_pool/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@vgg16_2/block1_pool/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](vgg16_2/block1_conv2/Relu, vgg16_2/block1_pool/MaxPool, training_1/Adam/gradients/vgg16_2/block2_conv1/convolution_grad/Conv2DBackpropInput)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_1/Adam/gradients/vgg16_2/block1_pool/MaxPool_grad/MaxPoolGrad', defined at:
  File "c:\users\garrick\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\garrick\ana